# Domain Specific Code Generation using the FormLang DSL

## Abstract

AI and LLM systems are being trained to perform a variety of complex tasks requiring expertise in many software tools, languages and technology stacks. Some of the challenges in this process involve quality data acquisition in large amounts, handling the increase of model parameter count which leads to increasing compute demands and costs, as well as emerging Data Privacy and Intellctual Property concerns related with using 3rd party cloud services for model training.

In this work we attempt to harness and combine the benefits of Abstraction and Determinism provided by Formal Domain Specific Languages (DSLs) with the innate ability of LLMs to learn new languages and their semantics. We propose a novel generation task called "Domain Specific Code Generation" which involve mapping user requests written in natural language to DSL code.

By utilizing a specially crafted DSL called `FormLang` as a case study, we attempt to lay the groundwork for methods of automated DSL dataset generation, training techniques and performance evaluation, with the end goal of creating an AI system capable of generating Web forms according to a user request. If successful and viable, these methods can be applied to other problem domains in which real-world processes can be formally modelled using DSLs, opening the door to the training of AI agents that understand these DSLs and can be engineered to perform tasks autonomously.

Our `FormLang` DSL allows expressing the semantics of Web-forms using a simplified syntax that does not require much, if any, Web-programming knowledge and expertise.

Given a user prompt in English describing the desired form and its fields the LLM produces syntactically valid `FormLang` output which is run through the accompanying FormLang parser and a hand-crafted React JSX compiler to output a final implementation of the form in JavaScript and React.

**(TBD)** The project includes a live demo which demonstrates the capabilities of the system.


## Referring to this work

If you use this work the following quote is preferred:

```bibtex
@misc{guyor2025dscodegenformlang,
      title={Domain Specific Code Generation using the FormLang DSL},
      author={Guy Or},
      year={2025}
}
```

The official repository of this work is hosted in GitHub at https://github.com/guyo13/Form-Lang.

## Project Goals

* Create an AI training pipeline for FormLang (implemented as a Juypter notebook) which includes:
    * Automatic FormLang Dataset generation using searching algorithms and heuristics.
    * Baseline model selection and loading from Hugging Face.
    * Dataset Preprocessing and loading.
    * Defining performance metrics for the model.
    * Model fine tuning using Transformers library
    * Model Adapter training using PEFT library.
    * Model upload to Hugging Face Hub and example usage from the Hub.
    *  **(TBD)** Export to ONNX using Optimum and run on-device using Transformers.js.
* Create the “FormLang” language:
    * Describe the problem domain.
    * Defining a viable minimal syntax and semantics which are research focused rather than completeness focused.
    *  **(TBD)** Implementing a “JavaScript React” compile target.
*  **(TBD)** Create a live demo website:
    * **(TBD)** Users input a prompt.
    * **(TBD)** A FormLang editor is populated with the AI’s code generation results.
    * **(TBD)** The form is rendered alongside the generated code.
*  Discuss the project results:
    * Perfornace and user acceptability.
    * **(TBD)** Viability of the implemented methods for the Domain Specific Code Generation task and generalization to other domains.  
    * **(TBD)** Potential enhancements to the system.
    * **(TBD)** Possible research directions on how to learn from user data.


**(TBD)** - Features marked as TBD are depending on the project's progress and timeline constraints as well as proving the viability of the methods and system.

# Notebook Setup

## About the FormLang DSL and its codebase

The FormLang DSL is implemented using [Langium](https://langium.org/). Langium is a toolkit written in TypeScript that allows language engineers to create DSLs and quickly iterate over their development lifecycle.

Some of the many features offered by Langium are:

* Parser generation - Langium leverages [Chevrotain](https://chevrotain.io/docs/) to create a fast parser that can parse the DSL code to AST.
* TypeScript AST nodes generation - Langium automatically generates TS types that represent the AST of the language.
* Simplified Validation rules - Langium has built in support for writing and executing validation rules with built in error reporting.
* Language Server Protocol support - Langium implements the LSP protocol which allows code completion and syntax highlighting in many IDEs such as VSCode.

Since we are using Langium to parse the FormLang DSL, some of this project tooling and algorithms are written in TypeScript.

To call our TypeScript code from Python we are using two distinct methods:

* [PythonMonkey](https://pythonmonkey.io/) - A Python package that implements FFI between Python and JavaScript.
* Local HTTP API - For some cases in which the PythonMonkey FFI is too slow or buggy, we've implemented a local REST-API server that implements some functions we require (e.g for parsing FormLang code to AST).

### Codebase

The entire codebase is hosted in Github at [FormLang](https://github.com/guyo13/Form-Lang).

### Running from Colab

#### Installing PNPM and Bun

In [1]:
!npm install -g pnpm
!npm install -g bun@latest
!npm install -g corepack@latest

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼
added 1 package in 4s
⠴
⠴1 package is looking for funding
⠴  run `npm fund` for details
⠴⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙
added 5 packages in 14s
⠙⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴
changed 1 package in 2s
⠴

In [2]:
!bun -v

1.2.2


#### Installing PythonMonkey

In [3]:
!pip install --quiet pythonmonkey

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.6/21.6 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.6/288.6 kB 19.0 MB/s eta 0:00:00


#### Installing Huggingface libraries and Unsloth

In [4]:
!pip install --quiet datasets evaluate bitsandbytes unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.6/187.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6

#### Installing dependencies for ROUGE score

In [5]:
!pip install --quiet nltk absl-py rouge_score

  Preparing metadata (setup.py) ... done


#### Installing dependencies for CHRF score

In [69]:
!pip install sacrebleu --quiet

#### Installing Flash attention 2

In [6]:
!pip install flash-attn --quiet --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 51.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


#### Installing Optimum-Quato

In [7]:
!pip install optimum-quanto --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.1/165.1 kB 4.9 MB/s eta 0:00:00


#### Clone the repo

In [8]:
!rm -rf Form-Lang

In [9]:
!git clone -b react_compiler https://github.com/guyo13/Form-Lang.git

Cloning into 'Form-Lang'...
remote: Enumerating objects: 737, done.
remote: Counting objects: 100% (157/157), done.
remote: Compressing objects: 100% (133/133), done.
remote: Total 737 (delta 113), reused 34 (delta 24), pack-reused 580 (from 1)
Receiving objects: 100% (737/737), 825.23 KiB | 10.58 MiB/s, done.
Resolving deltas: 100% (476/476), done.


#### Install project dependencies and build

In [10]:
!cd Form-Lang && corepack install && corepack pnpm install && corepack pnpm run gb

Adding pnpm@10.0.0+sha512.b8fef5494bd3fe4cbd4edabd0745df2ee5be3e4b0b8b08fa643aa3e4c6702ccc0f00d68fa8a8c9858a735a0032485a44990ed2810526c875e416f001b17df12b to the cache...
Lockfile is up to date, resolution step is skipped
Progress: resolved 1, reused 0, downloaded 0, added 0
Packages: +502

Progress: resolved 1, reused 0, downloaded 0, added 0
Progress: resolved 502, reused 0, downloaded 0, added 0

   ╭──────────────────────────────────────────────────────────────────╮
   │                                                                  │
   │                Update available! 10.0.0 → 10.4.1.                │
   │   Changelog: https://github.com/pnpm/pnpm/releases/tag/v10.4.1   │
   │         Run "corepack install -g pnpm@10.4.1" to update.         │
   │                                                                  │
   ╰──────────────────────────────────────────────────────────────────╯

Progress: resolved 502, reused 0, downloaded 0, added 0
Progress: resolved 502, reused 0, do

In [11]:
!cd Form-Lang/ml && pnpm install

Lockfile is up to date, resolution step is skipped
Progress: resolved 1, reused 0, downloaded 0, added 0
Packages: +69

Progress: resolved 1, reused 0, downloaded 0, added 0
Progress: resolved 69, reused 0, downloaded 0, added 0
Progress: resolved 69, reused 23, downloaded 0, added 0
Progress: resolved 69, reused 23, downloaded 20, added 33
Progress: resolved 69, reused 23, downloaded 46, added 69, done

dependencies:
+ express 4.21.2

Done in 1.5s


#### Run the FormLang HTTP API server

In [50]:
from subprocess import Popen, PIPE
import sys

def run_js_with_bun(js_file, cwd=None):
  """
  Runs a JavaScript file using Bun as a subprocess.

  Args:
    js_file: The path to the JavaScript file.
    cwd: The working directory for the child process.
         If None, the current working directory is used.
  """
  try:
    # Execute the JavaScript file using Bun with specified working directory
    process = Popen(['bun', js_file], stdout=PIPE, cwd=cwd)
    return process

  except FileNotFoundError:
    print("Error: Bun not found. Please ensure Bun is installed and in your PATH.")
  except Exception as e:
    import traceback
    traceback.print_exc()
    print("An unexpected error occurred:", e)

http_server = run_js_with_bun("lib_server.cjs", "Form-Lang/ml")
http_server

<Popen: returncode: None args: ['bun', 'lib_server.cjs']>

In [51]:
!curl -d '{"sourceCode": "component hey{} form helloWorld {comp hey}"}' -H "content-type: application/json" "http://localhost:3000/compute/ast"

{"status":"ok","result":{"ast":"{\"$type\":\"Form\",\"name\":\"helloWorld\",\"component\":{\"$type\":\"FieldComponentDef\",\"componentId\":{\"$ref\":\"#/components@0\"},\"componentPropsKeys\":[],\"componentPropsValues\":[]},\"children\":[]}"}}

## Project imports

In [14]:
import asyncio
import time
import re
import functools
import json
from pprint import pprint
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, GenerationConfig, QuantizedCacheConfig
from datasets import Dataset, load_dataset
from huggingface_hub import login
from tqdm.notebook import trange, tqdm
import evaluate
import pythonmonkey as pm
import pandas as pd
import requests
import torch
try:
    # If using locally from the `ml` folder
    formlang_lib = pm.require("../out/cjs/lib/index")
except:
    # If using in Colab:
    formlang_lib = pm.require("./Form-Lang/out/cjs/lib/index")

## Login to Huggingface Hub

In [15]:
login()

## Helper functions

### Python Monkey helpers

These can be used to print JavaScript objects that live inside the PythonMonkey engine.

In [16]:
def js_dir(something):
    pm.globalThis.console.dir(something)

def js_log(something):
    pm.globalThis.console.log(something)

### Computing the AST

In order to check the model's output we will need to parse the output FormLang code and check it for errors. We will also use the AST to compute performance metrics. Our `get_ast` function will be used to send FormLang source code to our local HTTP API for parsing.  

In [17]:
def get_ast(code, shouldCheckErrors=False):
    result = requests.post("http://localhost:3000/compute/ast", json={"sourceCode": code, "shouldCheckErrors":shouldCheckErrors}).json()
    if not result["status"] == "ok":
        raise RuntimeError("failed")
    return result["result"]

# Dataset generation

## Generating the dataset

The `ProbabilisticSearchFormGenerator` is class implemeting a parameterized algorithm for randomly generating forms:

* Iterate Depth-First, start with an empty form definition.
* For each item off the frontier if the item is a Field, append it to its Parent Form.
* If the item is a Form, generate its children and add them onto the frontier. Generate children according to the following rules:
    * Assign to the child node $depth = parentDepth + 1$.
    * There is a probability of $\alpha^{d}$ of generating a random child Form where $0 \leq \alpha < 1$ and $d$ is the depth of the child.
    * There is an optional parameter $D$ for defining the maximum depth for nested forms, which sets to $0$ the probability to generate a nested Form if its depth will be $D$.
    * There is a probability of $1 - \alpha^{d}$ of generating a random child Field.
    * The number of generated children is a random number in the range of $[minChildren, maxChildren]$.
    * There is a probability of $\beta$ that a generated Field will contain a `state`.
      * There is a probability of $\gamma$ that a state definition will be an `array`.
        * The number of array elements will be chosen at random from the integer interval $[amin, amax]$.
      * There is an equal probability of the state `type` to be any of the supported built-in types.
      * There is a probability of $\delta$ that a state definition will contain a `default` value.
        * If the `type` is `string` there is an $\epsilon$ probability of the default value to be defined as `as expression`, otherwise the probability is $1$.
* For each item off the frontier chose a random component from a set of available components.
  * Choose at random the number of assigned component props from the range $[0, ComponentPropCount]$.
    * For each assigned prop generate a random value with $\epsilon$ probability of the value to be defined as `as expression`

The algorithm ranomly generates the tree structure of the form which is then serialized into `FormLang` source code.

Some of the main advatages of using probabilistic search is the simplicity of implementation as well as the ability to create datasets representing different distributions by adjusting the generation hyper parameters.  

We start by creating a [ProbabilisticSearchFormGenerator](https://github.com/guyo13/Form-Lang/blob/react_compiler/src/generation/formGen.ts#L102) object which is implemented in JavaScript, using the default search hyper parameters and some hard-coded Component definitions.

Later on we will implement a random generator for the component definitions as well.

In [ ]:
form_components = """
  component userDetailsContainer {}
  component formContainer {}
  component someOtherContainer {}
  component OtherContainer2 {}
"""
field_components = """
  component myTextBox {
    props {
      textColor
      textSize
      textWeight
      borderColor
    }
  }
  component myCheckbox {
    props {
      size
    }
  }
  component otherTextBox {}
  component counter {
    props {
      style
    }
  }
"""

form_gen = await formlang_lib.newFormGen(formlang_lib.DEFAULT_GENERATOR_HYPER_PARAMETERS, form_components, field_components)

### Create a data generation loop for the prompt data

#### Prompt-Data generation algorithm

Each training example consists of a randomly generated form, from-which we remove (or "mask") a random node, keep track of the removed node's parent and siblings.

The goal being to create a prompt that provides the masked form and describes in English the node that needs to be added along with its location in the form.

The algoithm is as follows:

* Generate a random Form - outputs $F$.
* Serialize the Form to FormLang code - outputs $s(F)$.
* Iterate Depth-First starting at the form's root.
  * For every node the probability of removing the node is $1 - \zeta^{depth(node)}, (0 < \zeta < 1)$ if no node was removed, else the probability is $ 0 $.
  * If a node is marked to be removed, recording its surrounding context and remove it from the tree. - $F \to (F', N, ctx(F, N))$.
* Serialize the modified Form to FormLang code - outputs $s(F')$.
* Generate Prompt instruction parts:
* Serialize the removed child node into plain English - outputs $eng(N)$.
* Serialize the Parent and Siblings nodes ids and node types (form/field) into plain English - outputs $eng(ctx(F, N))$.
* Output a dict consisting of $s(F), s(F'), eng(N), eng(ctx(F, N))$

The algorithm is implemented in TypeScript in the [generateRandomFormWithModification](https://github.com/guyo13/Form-Lang/blob/react_compiler/src/lib/generator.ts#L43) function.

We create a small dataset of 3000 random examples by using the JavaScript FormLang utilities library and the form generator object.

We copy from the form generator's output only the relevant fields which will make up the LLM prompt.

For each generated example we also compute the original form's AST and store it as part of the dataset.

In [ ]:
# TODO - Refactor data generator pipeline as a Python class with easy interfaces

def concat_components_code_with_form_code(form_code, form_components, field_components):
    """Concatenates the form code with the given components code that are referenced inside of it."""
    return f"{form_components}\n{field_components}\n{form_code}"

def create_prompt_data(generation_result, form_components, field_components):
    data = {
        "originalFormCode": concat_components_code_with_form_code(generation_result["serializedForm"], form_components, field_components),
        "modifiedFormCode": concat_components_code_with_form_code(generation_result["serializedModifiedForm"], form_components, field_components),
        "removedNodeEnglish": str(generation_result["removedNodeEnglish"]),
        "removedNodeContextEnglish": str(generation_result["removedNodeContextEnglish"]),
    }
    return data

def create_random_data(form_gen, form_components, field_components, num_examples=3000):
    start_time = time.time()
    DATA = []
    for i in range(num_examples):
        generation_result = formlang_lib.generateRandomFormWithModification(form_gen)
        prompt_data = create_prompt_data(generation_result, form_components, field_components)
        DATA.append(prompt_data)
    # Generate the ast of each example in one batch
    ast_time = time.time()
    for example in DATA:
        resp = get_ast(example["originalFormCode"], True)
        errors = resp.get('errors', [])
        if len(errors) > 0:
            print(f"Error parsing generated code", errors)
            raise RuntimeError("Error parsing generated code")
        example["originalFormAst"] = resp["ast"]
    end_time = time.time()
    print(f"Generation took {end_time - start_time} seconds. Total Ast generation time {end_time - ast_time} seconds")
    return pd.DataFrame(DATA)

In [ ]:
examples = create_random_data(form_gen, form_components, field_components, 3000)
examples.head()

Generation took 48.5854971408844 seconds. Total Ast generation time 47.7074978351593 seconds


,originalFormCode,modifiedFormCode,removedNodeEnglish,removedNodeContextEnglish,originalFormAst
0,\n component userDetailsContainer {}\n compo...,\n component userDetailsContainer {}\n compo...,\ta field whose id is 'Z_' with state of type:...,* is a child of the form whose id is 'Bw'.\n* ...,"{""$type"":""Form"",""name"":""Bw"",""component"":{""$typ..."
1,\n component userDetailsContainer {}\n compo...,\n component userDetailsContainer {}\n compo...,\ta field whose id is 'p_' with state of type:...,* is a child of the form whose id is 'Cw'.\n* ...,"{""$type"":""Form"",""name"":""Cw"",""component"":{""$typ..."
2,\n component userDetailsContainer {}\n compo...,\n component userDetailsContainer {}\n compo...,\ta field whose id is 'F' using the component ...,* is a child of the form whose id is 'Vw'.\n* ...,"{""$type"":""Form"",""name"":""Vw"",""component"":{""$typ..."
3,\n component userDetailsContainer {}\n compo...,\n component userDetailsContainer {}\n compo...,\t\ta field whose id is 'Pww_ww___w' using the...,* is a child of the form whose id is 'd'.\n* i...,"{""$type"":""Form"",""name"":""Lw"",""component"":{""$typ..."
4,\n component userDetailsContainer {}\n compo...,\n component userDetailsContainer {}\n compo...,\ta field whose id is 'nw' with state of type:...,* is a child of the form whose id is 'a_'.\n* ...,"{""$type"":""Form"",""name"":""a_"",""component"":{""$typ..."


#### Example generated FormLang code and its AST

In [ ]:
examples["originalFormCode"][1132]

'\n  component userDetailsContainer {}\n  component formContainer {}\n  component someOtherContainer {}\n  component OtherContainer2 {}\n\n\n  component myTextBox {\n    props {\n      textColor\n      textSize\n      textWeight\n      borderColor\n    }\n  }\n  component myCheckbox {\n    props {\n      size\n    }\n  }\n  component otherTextBox {}\n  component counter {\n    props {\n      style\n    }\n  }\n\nform T_ {\n\tcomp userDetailsContainer \n\t\n\tform H {\n\t\tcomp userDetailsContainer \n\t\t\n\t\tfield Ew {\n\t\t\tcomp otherTextBox \n\t\t\t\n\t\t}\n\t\tfield Rw_w {\n\t\t\tcomp otherTextBox \n\t\t\t\n\t\t}\n\t\tfield F_ {\n\t\t\tstate number\n\t\t\tcomp myTextBox borderColor="#301410" textSize="(() => 97.65823852274963)()" as expression\n\t\t\t\n\t\t}\n\t}\n\n\tfield p___ {\n\t\tcomp otherTextBox \n\t\t\n\t}\n\tfield vw {\n\t\tstate string default "Frankie Kessler"\n\t\tcomp counter style="#5b426f"\n\t\t\n\t}\n\tfield f {\n\t\tcomp counter \n\t\t\n\t}\n\tfield q {\n\t\tstat

In [ ]:
examples["originalFormAst"][1132]

'{"$type":"Form","name":"T_","component":{"$type":"FieldComponentDef","componentId":{"$ref":"#/components@0"},"componentPropsKeys":[],"componentPropsValues":[]},"children":[{"$type":"Form","name":"H","component":{"$type":"FieldComponentDef","componentId":{"$ref":"#/components@0"},"componentPropsKeys":[],"componentPropsValues":[]},"children":[{"$type":"Field","name":"Ew","component":{"$type":"FieldComponentDef","componentId":{"$ref":"#/components@6"},"componentPropsKeys":[],"componentPropsValues":[]}},{"$type":"Field","name":"Rw_w","component":{"$type":"FieldComponentDef","componentId":{"$ref":"#/components@6"},"componentPropsKeys":[],"componentPropsValues":[]}},{"$type":"Field","name":"F_","state":{"$type":"FieldStateDef","type":"number","isArray":false},"component":{"$type":"FieldComponentDef","componentId":{"$ref":"#/components@4"},"componentPropsKeys":[{"$type":"ComponentPropKey","key":"borderColor"},{"$type":"ComponentPropKey","key":"textSize"}],"componentPropsValues":[{"$type":"Va

### Creating the user prompts

For each example in our synthetic dataset, we create the user prompt which will be used to train and test the LLM.

The user prompt consists of the modified FormLang code taken from each of our dataset examples along with the english description of the form element to add and its location (context) in the form.

In [ ]:
def create_prompt(row):
    return (
            "```FormLang\n"
            f"{row['modifiedFormCode']}\n"
            "```\n"
            "The description of the form element you need to add:\n"
            f"{row['removedNodeEnglish']}\n"
            "The description of the context in the form where you should add the element:\n"
            f"The element to be added:\n{row['removedNodeContextEnglish']}\n"
           )

In [ ]:
examples["userPrompt"] = examples.apply(create_prompt, axis=1)
examples.head()

,originalFormCode,modifiedFormCode,removedNodeEnglish,removedNodeContextEnglish,originalFormAst,userPrompt
0,\n component userDetailsContainer {}\n compo...,\n component userDetailsContainer {}\n compo...,\ta field whose id is 'Z_' with state of type:...,* is a child of the form whose id is 'Bw'.\n* ...,"{""$type"":""Form"",""name"":""Bw"",""component"":{""$typ...",```FormLang\n\n component userDetailsContaine...
1,\n component userDetailsContainer {}\n compo...,\n component userDetailsContainer {}\n compo...,\ta field whose id is 'p_' with state of type:...,* is a child of the form whose id is 'Cw'.\n* ...,"{""$type"":""Form"",""name"":""Cw"",""component"":{""$typ...",```FormLang\n\n component userDetailsContaine...
2,\n component userDetailsContainer {}\n compo...,\n component userDetailsContainer {}\n compo...,\ta field whose id is 'F' using the component ...,* is a child of the form whose id is 'Vw'.\n* ...,"{""$type"":""Form"",""name"":""Vw"",""component"":{""$typ...",```FormLang\n\n component userDetailsContaine...
3,\n component userDetailsContainer {}\n compo...,\n component userDetailsContainer {}\n compo...,\t\ta field whose id is 'Pww_ww___w' using the...,* is a child of the form whose id is 'd'.\n* i...,"{""$type"":""Form"",""name"":""Lw"",""component"":{""$typ...",```FormLang\n\n component userDetailsContaine...
4,\n component userDetailsContainer {}\n compo...,\n component userDetailsContainer {}\n compo...,\ta field whose id is 'nw' with state of type:...,* is a child of the form whose id is 'a_'.\n* ...,"{""$type"":""Form"",""name"":""a_"",""component"":{""$typ...",```FormLang\n\n component userDetailsContaine...


##### Example User Prompt vs. original Code

In [ ]:
print(examples['userPrompt'][0])

```FormLang

  component userDetailsContainer {}
  component formContainer {}
  component someOtherContainer {}
  component OtherContainer2 {}


  component myTextBox {
    props {
      textColor
      textSize
      textWeight
      borderColor
    }
  }
  component myCheckbox {
    props {
      size
    }
  }
  component otherTextBox {}
  component counter {
    props {
      style
    }
  }

form Bw {
	comp OtherContainer2 
	
	field G {
		comp myCheckbox size="'Hello'.toLowerCase()" as expression
		
	}
	form a {
		comp formContainer 
		
		field qw {
			state boolean[]
			comp otherTextBox 
			
		}
		field F__ {
			comp myTextBox 
			
		}
		field Y__ {
			state string default "'John Streich'" as expression
			comp myTextBox 
			
		}
		field S_w {
			state string
			comp myCheckbox size="#121310"
			
		}
		field N_ {
			state number default "0.17049162264892637" as expression
			comp counter 
			
		}
	}

	field pw_wwww_w__ {
		state boolean
		comp counter 
		
	}
	field J {
		state s

In [ ]:
print(examples['originalFormCode'][0])


  component userDetailsContainer {}
  component formContainer {}
  component someOtherContainer {}
  component OtherContainer2 {}


  component myTextBox {
    props {
      textColor
      textSize
      textWeight
      borderColor
    }
  }
  component myCheckbox {
    props {
      size
    }
  }
  component otherTextBox {}
  component counter {
    props {
      style
    }
  }

form Bw {
	comp OtherContainer2 
	
	field G {
		comp myCheckbox size="'Hello'.toLowerCase()" as expression
		
	}
	form a {
		comp formContainer 
		
		field qw {
			state boolean[]
			comp otherTextBox 
			
		}
		field F__ {
			comp myTextBox 
			
		}
		field Y__ {
			state string default "'John Streich'" as expression
			comp myTextBox 
			
		}
		field S_w {
			state string
			comp myCheckbox size="#121310"
			
		}
		field N_ {
			state number default "0.17049162264892637" as expression
			comp counter 
			
		}
	}

	field pw_wwww_w__ {
		state boolean
		comp counter 
		
	}
	field Z_ {
		state string
		com

### Creating a Dataset on HuggingFace using `Datasets`

First we've manually created a Dataset repository on the hugging-face hub for our [form-lang-examples](https://huggingface.co/datasets/guy-or/form-lang-examples) .

We then convert our `examples` DataFrame into a Dataset object and upload it to the Hub.


In [ ]:
ds = Dataset.from_pandas(examples)
ds.push_to_hub("guy-or/form-lang-examples", "3k_single_omission")
ds

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/30.0 [00:00<?, ?B/s]

Dataset({
    features: ['originalFormCode', 'modifiedFormCode', 'removedNodeEnglish', 'removedNodeContextEnglish', 'originalFormAst', 'userPrompt'],
    num_rows: 3000
})

# Zero-shot evaluation and Training using Llama 3.2-1B as a foundation model

We've selected Llama 3.2 1B Instruct as a foundantion model of our experimentation.

This is choice is driven by the popularity of the model and the abundace of resources and examples available online. Specifically the 1B version was chosen because it can run on consumer hardware and can be trained on free cloud resources.

Another important factor in this choice is Llama's large context size of 128K tokens, this allows us to experiment with large prompts and opens the door for building complex AI agents that can be fed with `FormLang`'s compiler messages and user feedback and adjust their responses.

Successfully demonstrating performance improvement after training the 1B version will justify further time and cost investment required for training larger models.

## Dataset Loading and Pre-processing

### Loading the Dataset from HuggingFace using `Datasets`

We load the Dataset from the Hub using `load_dataset` function. The dataset contains the raw user prompt and expected outputs.

In the next steps we will use these fields to create a full prompt which includes our system prompt and finally tokenize the prompt in preparation for running the model.

In [24]:
ds_3k_single_omission = load_dataset("guy-or/form-lang-examples", "3k_single_omission")
ds_3k_single_omission_all = ds_3k_single_omission['train']
ds_3k_single_omission_all

Dataset({
    features: ['originalFormCode', 'modifiedFormCode', 'removedNodeEnglish', 'removedNodeContextEnglish', 'originalFormAst', 'userPrompt'],
    num_rows: 3000
})

### Pre-processing using the Chat Template

In order to create prompts that effectively describe the task to our Llama 3.2 model, we need to format our prompt in a way that the model was trained to respond to.

This is called a [Prompt Template](https://www.llama.com/docs/model-cards-and-prompt-formats/llama3_1/#prompt-template), which consists of a **System prompt** which provides the initial context and guidelines for the model, followed by a **User prompt** containing the request and relevant data.

#### Loading the Llama Tokenizer

We load Llama's tokenizer using Huggingface `AutoTokenizer` class and leverage its [Chat templates](https://huggingface.co/docs/transformers/v4.48.2/en/chat_templating#introduction) capabilities which exposes a simple API for formatting our prompts into Llama's required Prompt Template.

In [22]:
# Setting the padding side to left for generation - later  on for training we will use right
llama3_tok_for_inference = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct", padding_side="left")
llama3_tok_for_inference.pad_token = "<|finetune_right_pad_id|>"

#### System prompt

Our System prompt instructs the LLM to generate the original form (the `originalFormCode` column) given the data present in the user prompt which we already created in the Dataset generation part.

In [20]:
def get_system_prompt():
    return (
        "You are a code generation AI assistant.\n"
        "Your job is to generate valid FormLang code according to the instructions given below:\n"
        "Inspect the following FormLang Form definition, the start of the code will be denoted with ```FormLang and its end with ``` .\n"
        "After inspection, complete the Form's code according to the given a description of a new form element and a description of its location in the form.\n"
        "You may assume that the new form element to be added is always either a 'form' or a 'field'.\n"
        "Your output must be valid and compiler-friendly FormLang code only.\n"
        "If you are unsure of the FormLang syntax, try to infer it from the form code which is given below as an input."
        "Your answer will be evaluated using an AST comparison of your code to the expected code.\n"
        "Assume that the input code is valid and requires no modification other than the NEW code you must generate.\n"
        "You must output plain FormLang code without any additional text or delimiters.\n"
        "You must not change any part of the original input code other than adding the required element.\n"
    )



#### Creating the full chat prompts

We create an `inferencePrompt` column to store the input prompt which contains the templated system and user prompts. **This column is used for inference only.**

Since our model's generated text gets appended (according to the prompt template format) to the input prompt, we also create a `chat` column containing the final expected model output. **This column will be used for training the model on a Causal Language Modelling task**. This task uses the model inputs (the `chat` column) shifted by one position to the right as labels.

In [23]:
def get_system_and_user_messages(userPrompt):
    return [
        {"role": "system", "content": get_system_prompt()},
        {"role": "user", "content": userPrompt}]

def get_full_conversation_messages(userPrompt, expectedOutput):
    messages = get_system_and_user_messages(userPrompt)
    messages.append({"role": "assistant", "content": expectedOutput})
    return messages

def preprocess_func(example, tokenizer):
    inference_prompt = tokenizer.apply_chat_template(get_system_and_user_messages(example["userPrompt"]), tokenize=False, add_generation_prompt=True)
    chat = tokenizer.apply_chat_template(get_full_conversation_messages(example["userPrompt"], example['originalFormCode']), tokenize=False, add_generation_prompt=False)
    return {"inferencePrompt": inference_prompt, "chat": chat}

In [52]:
# Our preprocess_func doesn't support batch operations, we pass in our Llama tokenizer as an argument
ds_3k_with_prompts = ds_3k_single_omission_all.map(preprocess_func, batched=False, fn_kwargs={"tokenizer": llama3_tok_for_inference})
ds_3k_with_prompts

Dataset({
    features: ['originalFormCode', 'modifiedFormCode', 'removedNodeEnglish', 'removedNodeContextEnglish', 'originalFormAst', 'userPrompt', 'inferencePrompt', 'chat'],
    num_rows: 3000
})

In [53]:
print(ds_3k_with_prompts[0]['inferencePrompt'])
print("#" * 50)
print(ds_3k_with_prompts[0]['chat'])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 17 Feb 2025

You are a code generation AI assistant.
Your job is to generate valid FormLang code according to the instructions given below:
Inspect the following FormLang Form definition, the start of the code will be denoted with ```FormLang and its end with ``` .
After inspection, complete the Form's code according to the given a description of a new form element and a description of its location in the form.
You may assume that the new form element to be added is always either a 'form' or a 'field'.
Your output must be valid and compiler-friendly FormLang code only.
If you are unsure of the FormLang syntax, try to infer it from the form code which is given below as an input.Your answer will be evaluated using an AST comparison of your code to the expected code.
Assume that the input code is valid and requires no modification other than the NEW code you must generate.
You mu

#### Creating the Train-Validation-Test split

We allocate 80% for training and 10% for test and validation each.

Since we are able to generate arbitrarily large amounts of data with different distributions, these percentages does not matter too much.

In [54]:
def create_train_validation_test_split(dataset, test_size):
    splits = dataset.train_test_split(test_size=test_size)
    test_splits = splits["test"].train_test_split(test_size=0.5) # Split so that test and validation sizes are equal
    return {"train": splits["train"], "test": test_splits["train"], "validation": test_splits["test"]}

ds_3k_splits = create_train_validation_test_split(ds_3k_with_prompts, 0.2)
ds_3k_train, ds_3k_test, ds_3k_val = ds_3k_splits["train"], ds_3k_splits["test"], ds_3k_splits["validation"]
ds_3k_train, ds_3k_test, ds_3k_val

(Dataset({
     features: ['originalFormCode', 'modifiedFormCode', 'removedNodeEnglish', 'removedNodeContextEnglish', 'originalFormAst', 'userPrompt', 'inferencePrompt', 'chat'],
     num_rows: 2400
 }),
 Dataset({
     features: ['originalFormCode', 'modifiedFormCode', 'removedNodeEnglish', 'removedNodeContextEnglish', 'originalFormAst', 'userPrompt', 'inferencePrompt', 'chat'],
     num_rows: 300
 }),
 Dataset({
     features: ['originalFormCode', 'modifiedFormCode', 'removedNodeEnglish', 'removedNodeContextEnglish', 'originalFormAst', 'userPrompt', 'inferencePrompt', 'chat'],
     num_rows: 300
 }))

## Post-Processing Utils

We create several utility functions for post processing the prompt data

### Extracting the model's response from the output text

Our model's raw output is a concatenation of the input and the generated text following Llama's Prompt Template semantics.

In order to extract the generated code we first need to identify the model's latest response by extracting all the text between the special `<|start_header_id|>assistant<|end_header_id|>` and `<|eot_id|>` tokens.

In [32]:
# prompt: Create a function that accepts the output text and extracts all the text starting at the LATEST OCCURRENCE of "<|start_header_id|>assistant<|end_header_id|>" and ends with "<|eot_id|>"

def extract_latest_assistant_response(text):
    """
    Extracts the model's response from the output text.

    Args:
        text: The output text from the model.

    Returns:
        The extracted model response, or None if no response is found.
    """
    match = re.findall(r"<\|start_header_id\|>assistant<\|end_header_id\|>(.*?)<\|eot_id\|>", text, re.DOTALL)
    if match:
        return match[-1].strip()  # Return the latest occurrence
    else:
        return None


### Extracting FormLang code from LLM output

The LLM might ignore the instruction to output plain code without the "```FormLang" delimiter, in that case we should try to extract the contents.

In [33]:
def extract_code_block(text, lang):
  """
  Extracts a code block from a string.

  Args:
    text: The string containing the code block.
    lang: The name of the language the code block is written in.

  Returns:
    The code block, or None if no code block is found.
  """
  if text is None:
    return None
  pattern = f"```{lang}\n(.*?)```"
  match = re.search(pattern, text, re.DOTALL)
  if match:
    return match.group(1).strip()
  return None

def extract_llm_formlang_output(text):
    # Strip leading and trailing whitespace and check for delimiters
    text = text.strip()
    assistant_response = extract_latest_assistant_response(text)
    return extract_code_block(assistant_response, "FormLang") or assistant_response

### Running inference on a single example

In [34]:
def clear_mem():
    """Clear unused GPU memory used by the python process"""
    import gc
    gc.collect()
    torch.cuda.empty_cache()

In [35]:
def run_inference(model, tok, example, cuda=True):
    """Runs pre-processing, inference and post-processing on a single example."""
    example_inputs = tok(example, return_tensors="pt")
    if cuda:
        example_inputs = example_inputs.to("cuda")
    with torch.no_grad():
      example_outputs = model.generate(**example_inputs, max_new_tokens=10000)
    generated_text = tok.decode(example_outputs[0], skip_special_tokens=False)
    del example_inputs
    del example_outputs
    generated_code = extract_llm_formlang_output(generated_text)
    ast = get_ast(generated_code)
    return {"generated_code": generated_code, "generated_text": generated_text, "ast": ast}

## Zero-shot evaluation of Llama 3.2 1B using Transformers and Evaluate

### Defining the Performance metrics

Before attempting to train the model, we want to define perfomance metrics for the code generation task and evaluate the base model performance on some of our data in order to establish a baseline for performance.

We will measure 3 metrics:

* Accuracy
* BLEU
* ROUGE

BLUE and ROUGE are standard metrics for measuring text generation performance when we want to compare the model's output to a desired output.

In the case of code generation, these metrics may not accurately reflect the quality of the model's output when applied to the source code and therefore we will try to apply them both on the serialized AST (in JSON format) of the generated code as well as the code itself and the raw output of the model vs the expected one.

**(TBD)** Try CodeBLEU and `apted` for tree edit distance algorithms that might better reflect the model's performance.

### Loading the metrics and creating evaluators

We load the evaluators for BLEU, ROUGE and Accuracy (using the "exact match" metric since our data is text).

In [55]:
def get_all_metrics(keep_in_memory=True):
    bleu_metric = evaluate.load("bleu", module_type="metric", keep_in_memory=keep_in_memory)
    rouge_metric = evaluate.load("rouge", module_type="metric", keep_in_memory=keep_in_memory)
    exact_match_metric = evaluate.load("exact_match", module_type="metric", keep_in_memory=keep_in_memory)
    all_metrics = evaluate.combine([bleu_metric, rouge_metric, exact_match_metric])
    return all_metrics

In [56]:
all_metrics = get_all_metrics()

#### Computing the metrics over our dataset columns

As an example to test the reliability of our metrics (specifically the BLEU and ROUGE scores), we check their average values on the examples from our train dataset.

We compare the similarity of the following pairs of data:

1) The expected raw output of the model (`chat`) compared to our `inferencePrompt`.

2) The original form source code (`originalFormCode`) and the modified form code (`modifiedFormCode`).

3) The AST of the original form source code and the AST of the modified form code.

In [68]:
def get_bleu_meteor_chrf(keep_in_memory=True):
    bleu_metric = evaluate.load("bleu", module_type="metric", keep_in_memory=keep_in_memory)
    meteor_metric = evaluate.load("meteor", module_type="metric", keep_in_memory=keep_in_memory)
    chrf_metric = evaluate.load("chrf", module_type="metric", keep_in_memory=keep_in_memory) # Output format {'score': 78.04486337325869, 'char_order': 6, 'word_order': 0, 'beta': 2}
    all_metrics = evaluate.combine([bleu_metric, meteor_metric, chrf_metric])
    return all_metrics

In [ ]:
%%time
get_bleu_meteor_chrf().compute(references=ds_3k_train["chat"], predictions=ds_3k_train["inferencePrompt"])

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [70]:
%%time
get_bleu_meteor_chrf().compute(references=ds_3k_train["originalFormCode"], predictions=ds_3k_train["modifiedFormCode"])

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


CPU times: user 15.5 s, sys: 471 ms, total: 16 s
Wall time: 17.8 s


{'bleu': 0.8764094193726288,
 'precisions': [1.0,
  0.9987436476187146,
  0.9953535096436337,
  0.9912785389372355],
 'brevity_penalty': 0.8796304707134517,
 'length_ratio': 0.8863257299356538,
 'translation_length': 374111,
 'reference_length': 422092,
 'meteor': 0.7956177186871536,
 'score': 91.9738319738464,
 'char_order': 6,
 'word_order': 0,
 'beta': 2}

In [71]:
%%time
get_bleu_meteor_chrf().compute(references=[json.dumps(originalFormAst) for originalFormAst in ds_3k_train["originalFormAst"]],
                    predictions=[json.dumps(get_ast(modifiedFormCode)) for modifiedFormCode in ds_3k_train["modifiedFormCode"]])

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


CPU times: user 1min 18s, sys: 1.54 s, total: 1min 20s
Wall time: 1min 57s


{'bleu': 0.8429359658014339,
 'precisions': [0.9991243684963839,
  0.9964290396238203,
  0.9945622915127816,
  0.9943585183190857],
 'brevity_penalty': 0.8462220852319774,
 'length_ratio': 0.8569175306304484,
 'translation_length': 2740879,
 'reference_length': 3198533,
 'meteor': 0.6904226543462031,
 'score': 87.7747112051328,
 'char_order': 6,
 'word_order': 0,
 'beta': 2}

In [57]:
%%time
# TODO - run concurrently using joblib
all_metrics.compute(references=ds_3k_train["chat"], predictions=ds_3k_train["inferencePrompt"])

CPU times: user 7min 18s, sys: 1.89 s, total: 7min 20s
Wall time: 8min 9s


{'bleu': 0.7184520643293014,
 'precisions': [1.0, 1.0, 1.0, 1.0],
 'brevity_penalty': 0.7184520643293014,
 'length_ratio': 0.7515088652201188,
 'translation_length': 1327336,
 'reference_length': 1766228,
 'rouge1': 0.8949557955579208,
 'rouge2': 0.8947424316459567,
 'rougeL': 0.8949688267843747,
 'rougeLsum': 0.8949925434276831,
 'exact_match': 0.0}

In [58]:
%%time
all_metrics.compute(references=ds_3k_train["originalFormCode"], predictions=ds_3k_train["modifiedFormCode"])

CPU times: user 42.5 s, sys: 258 ms, total: 42.7 s
Wall time: 43.1 s


{'bleu': 0.8764094193726288,
 'precisions': [1.0,
  0.9987436476187146,
  0.9953535096436337,
  0.9912785389372355],
 'brevity_penalty': 0.8796304707134517,
 'length_ratio': 0.8863257299356538,
 'translation_length': 374111,
 'reference_length': 422092,
 'rouge1': 0.9239790765548914,
 'rouge2': 0.9214753861355143,
 'rougeL': 0.9238912944045203,
 'rougeLsum': 0.9239619709063693,
 'exact_match': 0.0}

In [59]:
%%time
all_metrics.compute(references=[json.dumps(originalFormAst) for originalFormAst in ds_3k_train["originalFormAst"]],
                    predictions=[json.dumps(get_ast(modifiedFormCode)) for modifiedFormCode in ds_3k_train["modifiedFormCode"]])

CPU times: user 2min 17s, sys: 2.28 s, total: 2min 20s
Wall time: 2min 55s


{'bleu': 0.8429359658014339,
 'precisions': [0.9991243684963839,
  0.9964290396238203,
  0.9945622915127816,
  0.9943585183190857],
 'brevity_penalty': 0.8462220852319774,
 'length_ratio': 0.8569175306304484,
 'translation_length': 2740879,
 'reference_length': 3198533,
 'rouge1': 0.8370146985702014,
 'rouge2': 0.8334233528574165,
 'rougeL': 0.8369070588232945,
 'rougeLsum': 0.8370885974782589,
 'exact_match': 0.0}

As we can see, except for the `exact_match` score, our input is very "similar" to our expected output with the raw prompt comparison yielding scores above 50%.

This could mean on one hand that our trained model would have to exhibit scores that are very close to 100% in order to demonstrate a significant improvement,

and on the other hand scores that are far below the above scores are probably very bad even if they are above 50%.

We might also need to adjust our data generation algorithm to remove more nodes from the form (instead of a single omission algorithm for our example dataset).

In order to get a better sense of the baseline similarity in our dataset we plot a historam of the BLEU and ROUGE scores applied to the AST and the prompts and compute the medians on our test dataset.

In [ ]:
## TODO - Draw histogram plots and compute medians

In [ ]:
def evaluate_model(model, tokenizer, raw_text_evaluations, source_code_evaluations, ast_evaluations, dataset, batch_size=16):
  model.eval()
  all_predictions = []

  def evaluate_batch(batch):
    inputs = None
    outputs = None
    input_ids = None
    attention_mask = None
    try:
      inputs = tokenizer(batch["inferencePrompt"], return_tensors="pt", padding=True, truncation=False).to("cuda") # TODO pad_to_multiple_of=1024
      input_ids = inputs["input_ids"]
      attention_mask = inputs["attention_mask"]
      with torch.no_grad():
          t0 = time.time()
          print("generating")
          outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask)
          print(f"batch generated. took {time.time() - t0} seconds")

      # Evaluate output raw text vs. expected raw text
      raw_text_predictions = tokenizer.batch_decode(outputs, skip_special_tokens=True) # For the raw text evaluation we do skip special tokens because we want to be agnostic of any padding
      raw_text_references = tokenizer.batch_decode(tokenizer(batch["chat"], return_tensors="pt", padding=True, truncation=False).input_ids, skip_special_tokens=True) # Create full expected text references by removing special tokens
      raw_text_evaluations.add_batch(predictions=raw_text_predictions, references=raw_text_references)

      # Evaluate the source code output
      raw_llm_predictions = tokenizer.batch_decode(outputs, skip_special_tokens=False) # We dont skip special tokens because we need them to know how to extract the LLMs latest response
      source_code_predictions = [extract_llm_formlang_output(output) or "" for output in raw_llm_predictions]
      source_code_references = [original_code for original_code in batch["originalFormCode"]]
      source_code_evaluations.add_batch(predictions=source_code_predictions, references=source_code_references)

      # Evaluate the AST output - TODO Deal with errors
      #ast_predictions = [json.dumps(get_ast(code)) for code in form_lang_predictions]
      #ast_references = [ast for ast in batch["originalFormAst"]]
      #ast_evaluations.add_batch(predictions=ast_predictions, references=ast_references)

      # Add the llm outputs to the list and return it from the function
      all_predictions.extend(raw_llm_predictions)
      print("added to eval")
    except:
      print("got exception")
      import traceback
      traceback.print_exc()
    finally:
      # Free GPU memory
      del inputs
      del outputs
      del input_ids
      del attention_mask

  for batch in tqdm(dataset.iter(batch_size)):
    evaluate_batch(batch)

  return raw_text_evaluations, source_code_evaluations, ast_evaluations, all_predictions


### Zero-shot evaluation with the original Meta version

We the original Llama 3.2 1B model available from Meta and run an evaluation over our validation dataset

##### Initialize the model

In [ ]:
llama3_model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.2-1B-Instruct",
    torch_dtype=torch.bfloat16,
    device_map="cuda",
    attn_implementation="flash_attention_2"
)

##### Override default generation config

In [ ]:
llama3_model.generation_config.pad_token_id = llama3_tok_for_inference.pad_token_id
llama3_model.generation_config.temperature = 0.9
llama3_model.generation_config.max_new_tokens = 1024
print(llama3_model.generation_config)

GenerationConfig {
  "bos_token_id": 128000,
  "do_sample": true,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "max_new_tokens": 1024,
  "pad_token_id": 128004,
  "temperature": 0.9,
  "top_p": 0.9
}



##### [EXPERIMENT] Trying with static cache and torch compile

Only supported when `attn_implementation="sdpa"` - **slower than FA2**

In [ ]:
#llama3_model.generation_config.cache_implementation = "static"

#llama3_model.forward = torch.compile(llama3_model.forward, mode="reduce-overhead", fullgraph=True)

##### [EXPERIMENT] Trying with Quantized Cache

**Increases latency relative to the default DynamicCache**

In [ ]:
# Configure Quantized Cache
quantized_cache_config = QuantizedCacheConfig(
    nbits=4,  # Use 4-bit quantization
    q_group_size=64,  # Adjust if necessary
    residual_length=128,  # Adjust if necessary
)

#llama3_model.generation_config.cache_implementation = "quantized"
#llama3_model.generation_config.quantized_cache_config = quantized_cache_config

##### Trying to evaluate the first 50 examples in our validation dataset

In [ ]:
first_examples = ds_3k_val.train_test_split(test_size=(50/300))["test"]

In [ ]:
raw_text_evaluations, source_code_evaluations, ast_evaluations, all_predictions = evaluate_model(llama3_model, tok, get_all_metrics(), get_all_metrics(), get_all_metrics(), first_examples, batch_size=50)

0it [00:00, ?it/s]

generating
batch generated. took 75.79425168037415 seconds
added to eval


In [ ]:
llama3_tok_for_inference(all_predictions, return_tensors="pt", padding=True, truncation=False).input_ids.shape

torch.Size([50, 1559])

In [ ]:
print(all_predictions[42])

<|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|>

In [ ]:
print(extract_llm_formlang_output(all_predictions[42]))

component userDetailsContainer {}
  component formContainer {}
  component someOtherContainer {}
  component OtherContainer2 {}

  component myTextBox {
    props {
      textColor
      textSize
      textWeight
      borderColor
    }
  }
  component myCheckbox {
    props {
      size
    }
  }
  component otherTextBox {}
  component counter {
    props {
      style
    }
  }

  component gw {
    comp OtherContainer2 
    field id="qw" type="number"
    props textColor="#2f5276"
    textSize="#523067"
    textWeight="#383061"
    borderColor="#2f5660"
  }


In [ ]:
print(extract_llm_formlang_output(first_examples[42]["chat"]))

component userDetailsContainer {}
  component formContainer {}
  component someOtherContainer {}
  component OtherContainer2 {}


  component myTextBox {
    props {
      textColor
      textSize
      textWeight
      borderColor
    }
  }
  component myCheckbox {
    props {
      size
    }
  }
  component otherTextBox {}
  component counter {
    props {
      style
    }
  }

form gw {
	comp OtherContainer2 
	
	field qw {
		state number
		comp myTextBox textColor="#2f5276" textSize="#523067" textWeight="#383061" borderColor="#2f5660"
		
	}
}


In [ ]:
raw_text_evaluations.compute()

{'bleu': 0.9003100125972453,
 'precisions': [0.919347984458475,
  0.9050586733142822,
  0.8935269760077944,
  0.8836982374824663],
 'brevity_penalty': 1.0,
 'length_ratio': 1.0514154405897935,
 'translation_length': 32944,
 'reference_length': 31333,
 'rouge1': 0.9482152254161769,
 'rouge2': 0.9333667894401422,
 'rougeL': 0.93519168962034,
 'rougeLsum': 0.9484624191071347,
 'exact_match': 0.0}

In [ ]:
source_code_evaluations.compute()

{'bleu': 0.7925345856532497,
 'precisions': [0.9052563270603504,
  0.8615284128020901,
  0.8197238658777121,
  0.7843812045003309],
 'brevity_penalty': 0.9418012026122038,
 'length_ratio': 0.9434308803722297,
 'translation_length': 7705,
 'reference_length': 8167,
 'rouge1': 0.8594792255623867,
 'rouge2': 0.8006840527857084,
 'rougeL': 0.7981545069180424,
 'rougeLsum': 0.8586316529420399,
 'exact_match': 0.0}

In [ ]:
# TODO
#ast_evaluations.compute()

In [ ]:
#del llama3_model
#clear_mem()

### Zero-shot evaluation with Unsolth's 4-bit Quantized version of the model

In [ ]:
unsloth4_llama3 = AutoModelForCausalLM.from_pretrained("unsloth/Llama-3.2-1B-bnb-4bit", device_map="cuda")

config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


model.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

In [ ]:
%%time
uns_raw_text_evaluations, uns_source_code_evaluations, uns_ast_evaluations, uns_all_predictions = evaluate_model(unsloth4_llama3, llama3_tok_for_inference, get_all_metrics(), get_all_metrics(), get_all_metrics(), first_examples, batch_size=5)

0it [00:00, ?it/s]

generating
batch generated. took 47.3324236869812 seconds
decoded
added to eval
generating
batch generated. took 47.25919985771179 seconds
decoded
added to eval
generating


KeyboardInterrupt: 

In [ ]:
del unsloth4_llama3
clear_mem()

## Fine-Tuning LLama 3.2 1B

# Citations

TODO